In [ ]:
import cv2
import random
import numpy
import math
import statistics
from PIL import Image, ImageOps
from tqdm import tqdm
import scipy.linalg
from collections import Counter
from scipy.spatial import distance
from numpy.linalg import norm
import warnings
warnings.simplefilter("ignore", RuntimeWarning)

In [ ]:
img_read = Image.open(r"Data_mean_image.jpeg") 
count_rows, count_cols = 256, 256
count_pixels = count_rows * count_cols
img_read = img_read.resize((count_rows, count_cols))

img_array = numpy.array(img_read)

red_matrix = img_array[:,:,0]
red_max = numpy.amax(red_matrix)

green_matrix = img_array[:,:,1]
green_max = numpy.amax(green_matrix)

blue_matrix = img_array[:,:,2]
blue_max = numpy.amax(blue_matrix)

#Making a numpy array of all RGB vectors

list_vectors = []
for i in range(count_rows):
    for j in range(count_cols):
        list_pts = [img_array[i][j][0] / red_max, img_array[i][j][1] / green_max, img_array[i][j][2] / blue_max]
        list_pts.append(j / 255)
        list_pts.append(i / 255)
        list_vectors.append(list_pts)
        
np_pix_vectors = numpy.array(list_vectors)


In [ ]:
value_bandwidth = 0.2
value_threshold = 0.00001
percent_labelled = 0.85

count_pixels_threshold = percent_labelled * count_pixels
#Picking a random data point and starting the process
dict_pix_labels = {}
list_labelled = []
dict_history = {}
count_converged = 0
while True:
    count_prev_labelled = len(list_labelled)
    idx_initial_cluster = random.randint(0,count_pixels-1)
    print(f"index chosen = {idx_initial_cluster}")
    cluster_vector = np_pix_vectors[idx_initial_cluster]
    while True:
        #Now finding all datapoints within a distance of bandwidth
        list_vectors_included = []
        list_pix_idx_incl = []

        for i in range(np_pix_vectors.shape[0]):
            curr_pix_vector = np_pix_vectors[i]
            value_distance =  distance.euclidean(cluster_vector, curr_pix_vector)
            if value_distance <= value_bandwidth:
                list_pix_idx_incl.append(i)
                list_vectors = curr_pix_vector.tolist()
                list_vectors_included.append(list_vectors)
                dict_history[i] = cluster_vector[:3]

        #Computing the new mean
        np_list_vec_inc = numpy.array(list_vectors_included)
        vector_mean = numpy.mean(np_list_vec_inc, axis = 0)
        dist_bw_means = distance.euclidean(cluster_vector, vector_mean)
        
        if dist_bw_means <= value_threshold:
            count_converged += 1
            temp_colours = vector_mean[:3]
            list_colors = list(temp_colours)
            for a in list_pix_idx_incl:
                list_labelled.append(a)
                dict_pix_labels[a] = list_colors

            list_labelled = list(set(list_labelled))
            break
        cluster_vector = vector_mean
    
    if len(list_labelled) >= count_pixels_threshold:
        break

In [ ]:
list_pix_cord = []

for t in range(count_rows):
    for u in range(count_cols):
        list_pix_cord.append([t,u])

copy_img_arr = numpy.copy(img_array)
for pix in dict_pix_labels:
    list_colors = dict_pix_labels[pix]
    pix_cord = list_pix_cord[pix]
    x, y = pix_cord[0], pix_cord[1]
    copy_img_arr[x][y][0] = list_colors[0] * red_max
    copy_img_arr[x][y][1] = list_colors[1] * green_max
    copy_img_arr[x][y][2] = list_colors[2] * blue_max

In [ ]:
new_img_mean = Image.fromarray(copy_img_arr)
new_img_mean.save("Result_mean_segment.jpeg")
new_img_mean.show()